In [ ]:
!pip install catboost

: 

# CatBoost Algorithm

CatBoost is a state-of-the-art-open-source gradient boosting on decision trees library. It's simple and easy to use. And is no regularly one of the top algorithms used in data science competitions as it produces very good results without extensive data clean-up or feature engineering.

In [2]:
# import libraries
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier

In [ ]:
# data import from titanic
df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# Preprocessing


In [ ]:
# impute missing value using knn imputers in fare, age, embark and embark_town
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 5)

df['age'] = imputer.fit_transform(df[['age']])
df['fare'] = imputer.fit_transform (df[['fare']])

# impute using pandas 
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])
df['embark_town'] = df['embark_town'].fillna(df['embark_town'].mode()[0])

# drop deck clumn
df.drop('deck', axis =1 , inplace = True)

# df mising value
df.isnull().sum()


survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     891 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  embark_town  891 non-null    object  
 12  alive        891 non-null    object  
 13  alone        891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 79.4+ KB


In [ ]:
# convert each category column to category

for col in categorical_cols:
    df[col] = df[col].astype('category')


In [ ]:
# split the data into x and y
x = df.drop('survived', axis =1)
y = df['survived']

# split the data into train and test
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y , test_size = 0.2, random_state = 42)

# run the catboost classifier
model = CatBoostClassifier(iterations = 100,
                          learning_rate = 0.1,
                          depth = 3,
                           loss_function = 'Logloss',
                          eval_metric = 'accuracy',
                          random_state = 42,
                          verbose = False)

# train the model
model.fit(x_train, y_train, cat_features = categorical_cols.tolist())

# predicion
y_pred = model.predict(x_test)

# evaluate the model
print("accuracy", accuracy(y_train, y_pred))
print("classification Report\n", classification_report(y_train, y_pred))

CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/tools/enum_parser/enum_serialization_runtime/enum_runtime.cpp:70: Key 'accuracy' not found in enum ELossFunction. Valid options are: 'Logloss', 'CrossEntropy', 'CtrFactor', 'Focal', 'RMSE', 'LogCosh', 'Lq', 'MAE', 'Quantile', 'MultiQuantile', 'Expectile', 'LogLinQuantile', 'MAPE', 'Poisson', 'MSLE', 'MedianAbsoluteError', 'SMAPE', 'Huber', 'Tweedie', 'Cox', 'RMSEWithUncertainty', 'MultiClass', 'MultiClassOneVsAll', 'PairLogit', 'PairLogitPairwise', 'YetiRank', 'YetiRankPairwise', 'QueryRMSE', 'QuerySoftMax', 'QueryCrossEntropy', 'StochasticFilter', 'LambdaMart', 'StochasticRank', 'PythonUserDefinedPerObject', 'PythonUserDefinedMultiTarget', 'UserPerObjMetric', 'UserQuerywiseMetric', 'R2', 'NumErrors', 'FairLoss', 'AUC', 'Accuracy', 'BalancedAccuracy', 'BalancedErrorRate', 'BrierScore', 'Precision', 'Recall', 'F1', 'TotalF1', 'F', 'MCC', 'ZeroOneLoss', 'HammingLoss', 'HingeLoss', 'Kappa', 'WKappa', 'LogLikelihoodOfPrediction'

In [ ]:
# Import necessary libraries
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Assume df is your DataFrame and categorical_cols is defined
# df = pd.read_csv('your_data.csv') # Example of loading data
# categorical_cols = ['col1', 'col2', 'col3'] # Example of categorical columns

# Convert categorical columns to 'category' dtype
for col in categorical_cols:
    df[col] = df[col].astype('category')

# Split the data into x and y
x = df.drop('survived', axis=1)
y = df['survived']

# Split the data into train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Convert categorical column names to indices
cat_features_indices = [x.columns.get_loc(col) for col in categorical_cols]

# Run the CatBoost classifier
model = CatBoostClassifier(iterations=100,
                           learning_rate=0.1,
                           depth=3,
                           loss_function='Logloss',
                           eval_metric='accuracy',
                           random_state=42,
                           verbose=False)

# Train the model
model.fit(x_train, y_train, cat_features=cat_features_indices)

# Prediction
y_pred = model.predict(x_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report\n", classification_report(y_test, y_pred))


CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/tools/enum_parser/enum_serialization_runtime/enum_runtime.cpp:70: Key 'accuracy' not found in enum ELossFunction. Valid options are: 'Logloss', 'CrossEntropy', 'CtrFactor', 'Focal', 'RMSE', 'LogCosh', 'Lq', 'MAE', 'Quantile', 'MultiQuantile', 'Expectile', 'LogLinQuantile', 'MAPE', 'Poisson', 'MSLE', 'MedianAbsoluteError', 'SMAPE', 'Huber', 'Tweedie', 'Cox', 'RMSEWithUncertainty', 'MultiClass', 'MultiClassOneVsAll', 'PairLogit', 'PairLogitPairwise', 'YetiRank', 'YetiRankPairwise', 'QueryRMSE', 'QuerySoftMax', 'QueryCrossEntropy', 'StochasticFilter', 'LambdaMart', 'StochasticRank', 'PythonUserDefinedPerObject', 'PythonUserDefinedMultiTarget', 'UserPerObjMetric', 'UserQuerywiseMetric', 'R2', 'NumErrors', 'FairLoss', 'AUC', 'Accuracy', 'BalancedAccuracy', 'BalancedErrorRate', 'BrierScore', 'Precision', 'Recall', 'F1', 'TotalF1', 'F', 'MCC', 'ZeroOneLoss', 'HammingLoss', 'HingeLoss', 'Kappa', 'WKappa', 'LogLikelihoodOfPrediction'